<img src="../media/LandingPage-Header-RED-CENTRE.jpg" alt="Notebook Banner" style="width:100%; height:auto; display:block; margin-left:auto; margin-right:auto;">

# MLflow

## Simple Model Deployment and Evaluation

Let's walk through a basic workflow demonstrating how MLflow can be used to track experiments, log parameters, metrics, and models for a simple classification task.

### What is MLflow?

MLflow is an open-source platform for managing the end to end machine learning lifecycle. It addresses four primary functions:

* **Tracking:** Record and query experiments (code, data, config, results).

* **Projects:** Package ML code in a reusable, reproducible form.

* **Models:** Manage and deploy models from various ML libraries to diverse serving platforms.

* **Model Registry:** Centralized model store to collaboratively manage the full lifecycle of MLflow Models.

### Purpose of This Example

This example demonstrates a basic MLflow workflow for a binary classification problem using a Decision Tree Classifier. It covers:

1. **Setting up an MLflow Experiment:** Organizing runs under a common name.

2. **Starting an MLflow Run:** Creating a specific instance of an experiment.

3. **Data Preparation:** Generating synthetic data for a simplified binary classification.

4. **Logging Parameters:** Recording key data split parameters and model hyperparameters.

5. **Model Training:** Training a `DecisionTreeClassifier` on the prepared data.

6. **Calculating and Logging Metrics:** Evaluating the model's performance (accuracy, precision, recall, F1-score) and logging these metrics.

7. **Logging the Model:** Saving the trained model as an MLflow artifact, making it ready for deployment.

This setup allows for easy comparison of different runs, understanding the impact of various parameters, and reproducing results.

### Installation & Setup

First, you'd install MLflow in your Python environment:

* `pip install mlflow`

Lets review a simple classification example where we create some data and applies a decision tree.


In [2]:
import mlflow
import mlflow.sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import pandas as pd

# --- 1. Set the MLflow Experiment ---
mlflow.set_experiment("Decision_Tree_Classification_Analysis_Simplified")

# --- 2. Start an MLflow Run ---
with mlflow.start_run(run_name="Decision_Tree_Trial_Simplified_Data"):
    
    # --- Data Preparation for Binary Classification ---
    np.random.seed(42) # For reproducibility of random data

    # Create two simple features
    X = pd.DataFrame({
        'feature_1': np.random.rand(100) * 10,  # Random values between 0 and 10
        'feature_2': np.random.rand(100) * 10
    })

    # Create a simplified binary target:
    # If feature_1 is high (e.g., > 5), assign class 1. Otherwise, assign class 0.
    # This creates a clear, easy-to-understand separation for the target variable.
    y = (X['feature_1'] > 5).astype(int) 
    # Add a little noise to 'y' to make it less perfectly separable, simulating real-world data
    # We'll flip a small percentage of labels
    noise_idx = np.random.choice(len(y), size=10, replace=False) # Select 10 random indices
    y.iloc[noise_idx] = 1 - y.iloc[noise_idx] # Flip the labels at these indices

    # --- Log Data Split Parameters ---
    # These are crucial for understanding how the data was split for training/testing.
    test_split_ratio = 0.20 # 20% of data for testing
    is_stratified_split = True # Indicates if the split maintained class proportions

    mlflow.log_param("data_test_split_ratio", test_split_ratio)
    mlflow.log_param("data_is_stratified_split", is_stratified_split)
    
    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, 
        test_size=test_split_ratio, 
        random_state=42, 
        stratify=y if is_stratified_split else None # Apply stratification based on our new parameter
    )

    # --- Log Model Parameters ---
    max_depth_param = 4       # Max depth of the tree
    min_samples_leaf_param = 3 # Minimum samples required at a leaf node
    criterion_param = "entropy"  # Information gain for splitting
    
    mlflow.log_param("model_max_depth", max_depth_param)
    mlflow.log_param("model_min_samples_leaf", min_samples_leaf_param)
    mlflow.log_param("model_criterion", criterion_param)
    mlflow.log_param("input_features_count", X.shape[1])

    # --- Train a Decision Tree model ---
    model = DecisionTreeClassifier(max_depth=max_depth_param, 
                                   min_samples_leaf=min_samples_leaf_param,
                                   criterion=criterion_param,
                                   random_state=42)
    model.fit(X_train, y_train)

    # --- Make predictions and calculate metrics ---
    predictions = model.predict(X_test)
    
    # --- Log Metrics ---
    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, zero_division=0)
    recall = recall_score(y_test, predictions, zero_division=0)
    f1 = f1_score(y_test, predictions, zero_division=0)

    mlflow.log_metric("evaluation_accuracy", accuracy)
    mlflow.log_metric("evaluation_precision", precision)
    mlflow.log_metric("evaluation_recall", recall)
    mlflow.log_metric("evaluation_f1_score", f1)

    # --- 5. Log the Model (as an Artifact) ---
    mlflow.sklearn.log_model(model, "decision_tree_classifier_model")

    print(f"MLflow Run completed for Experiment '{mlflow.active_run().info.experiment_id}' and Run '{mlflow.active_run().info.run_id}'")
    print(f"Logged Data Split: Test Ratio={test_split_ratio}, Stratified={is_stratified_split}")
    print(f"Logged Model Parameters: Max Depth={max_depth_param}, Min Samples Leaf={min_samples_leaf_param}, Criterion={criterion_param}")
    print(f"Logged Metrics: Accuracy={accuracy:.4f}, Precision={precision:.4f}, Recall={recall:.4f}, F1-Score={f1:.4f}")
    print(f"Model saved as artifact: decision_tree_classifier_model")
    print(f"To view this run, run 'mlflow ui' in your terminal in this directory.")

2025/07/04 12:07:36 INFO mlflow.tracking.fluent: Experiment with name 'Decision_Tree_Classification_Analysis_Simplified' does not exist. Creating a new experiment.
2025/07/04 12:07:37 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/04 12:07:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


MLflow Run completed for Experiment '707572921101383408' and Run '60f5b250b43f432ea9b01fd51fb1762b'
Logged Data Split: Test Ratio=0.2, Stratified=True
Logged Model Parameters: Max Depth=4, Min Samples Leaf=3, Criterion=entropy
Logged Metrics: Accuracy=0.8000, Precision=0.7500, Recall=0.7500, F1-Score=0.7500
Model saved as artifact: decision_tree_classifier_model
To view this run, run 'mlflow ui' in your terminal in this directory.


**How to see your results: The MLflow UI**
- After running this script, open your terminal in the directory where your 'mlruns' folder is located.
- Then, execute:
    - `mlflow ui`
    - This will launch the MLflow Tracking UI, usually accessible at http://localhost:5000.
    - You'll be able to navigate to the "Decision_Tree_Classification_Analysis" experiment and inspect the details of "Decision_Tree_Trial_1", including its parameters, metrics, and the saved model.

### MLflow Tracking Cheat Sheet: Essential Logging Functions

Here a table that summarises the most common and essential `mlflow.log_*` functions, along with their purpose and examples. This will serve as a quick reference for effectively leveraging MLflow Tracking.

| MLflow Function | Purpose | When to Use | Example |
| :--------------------------- | :------------------------------------------------------------------------------------------------------ | :------------------------------------------------------------------- | :----------------------------------------------------------------------------------------------------------------------- |
| **`mlflow.log_param(key, value)`** | **Logs a single key-value parameter.** Values are typically strings, numbers, or booleans. Used for hyperparameters, configuration settings, or data characteristics. | To record **input settings** or **configurations** that define a specific run. Essential for reproducibility and comparison. | `mlflow.log_param("learning_rate", 0.01)` <br> `mlflow.log_param("optimizer", "Adam")` <br> `mlflow.log_param("data_scaling_method", "MinMaxScaler")` |
| **`mlflow.log_metric(key, value)`** | **Logs a single key-value metric.** Values must be numeric. MLflow tracks metrics as they change over time (e.g., during training epochs). | To record **quantitative performance indicators** of your model or experiment. Essential for comparing run performance. | `mlflow.log_metric("accuracy", 0.925)` <br> `mlflow.log_metric("validation_loss", 0.15, step=10)` (for epoch-based logging) <br> `mlflow.log_metric("f1_score", 0.88)` |
| **`mlflow.log_artifact(local_path, artifact_path=None)`** | **Logs a single file as an artifact.** The file is copied from `local_path` to the run's artifact URI. | To save **important output files** that are not parameters or metrics but provide context or are useful for later analysis/reproducibility. | `mlflow.log_artifact("plots/roc_curve.png")` <br> `mlflow.log_artifact("my_dataset_sample.csv", artifact_path="data")` |
| **`mlflow.log_artifacts(local_dir, artifact_path=None)`** | **Logs an entire directory of files as artifacts.** All files within `local_dir` are recursively copied. | To save **multiple related output files** at once (e.g., all plots from a run, all config files). | `mlflow.log_artifacts("results_folder")` <br> `mlflow.log_artifacts("configs/", artifact_path="model_configs")` |
| **`mlflow.<flavor>.log_model(model, artifact_path, ...)`** | **Logs a machine learning model specific to a framework/library (flavor).** This function packages the model along with necessary metadata, code dependencies, and often a signature. | To save your **trained ML model** in a standardized format that MLflow can later reload, serve, and deploy using various tools. **This is crucial for model management!** | `mlflow.sklearn.log_model(my_sklearn_model, "my_classification_model")` <br> `mlflow.pytorch.log_model(my_torch_model, "pytorch_model")` <br> `mlflow.tensorflow.log_model(my_tf_model, "tf_model")` |
| **`mlflow.set_experiment(experiment_name)`** | **Sets the current active experiment.** If the experiment doesn't exist, it's created. All subsequent runs will belong to this experiment. | To **organize your runs** into logical groups. This is typically one of the first lines of MLflow code in a script. | `mlflow.set_experiment("Customer_Churn_Prediction")` <br> `mlflow.set_experiment("Image_Recognition_CNN_Trials")` |
| **`mlflow.set_tag(key, value)`** | **Adds a tag (key-value pair) to the current run.** Tags are flexible strings used for arbitrary metadata. | To add **descriptive or categorizing metadata** that doesn't fit into parameters or metrics (e.g., developer name, data version, specific algorithm variant). | `mlflow.set_tag("developer", "Alice")` <br> `mlflow.set_tag("data_version", "v2.1_cleaned")` <br> `mlflow.set_tag("model_family", "tree_based")` |
| **`mlflow.start_run(run_name=None, ...)`** | **Starts an MLflow run and returns an `ActiveRun` object.** Typically used with a `with` statement for automatic run termination. | To **begin tracking a new experiment run.** This is the core context manager for capturing run information. | `with mlflow.start_run(run_name="Hyperparam_Search_Trial_1"):` <br> `  # Your ML code here` |


# MLflow: Multiple Models Comparison

## Comparing Different Classification Models on the Iris Dataset

This section demonstrates how to use MLflow to track and compare the performance of multiple machine learning models within a single experiment. We will use the classic **Iris dataset** for a multi-class classification problem.

### Purpose of This Example

The primary goal of this example is to illustrate MLflow's capability to facilitate model comparison by logging distinct runs for different models under a unified experiment. Specifically, it covers:

1.  **Experiment Setup:** Defining a new MLflow experiment (`Iris Classification Models`) to group all runs related to Iris classification.
2.  **Dataset Loading and Splitting:** Loading the Iris dataset and splitting it into training and testing sets.
3.  **Iterative Model Training and Logging:** Training three different classification models:
    * **K-Nearest Neighbors (KNN)**
    * **Support Vector Machine (SVM)**
    * **Decision Tree Classifier**
4.  **Parameter Tracking:** For each model, relevant hyperparameters (e.g., `n_neighbors` for KNN, `C` and `kernel` for SVM, `max_depth` for Decision Tree) are logged using `mlflow.log_param()`.
5.  **Metric Logging:** Standard classification metrics (Accuracy, Precision, Recall, F1-Score) are calculated on the test set and logged for each model using `mlflow.log_metric()`. The `average='weighted'` parameter is used for precision, recall, and F1 score to handle the multi class nature of the Iris dataset.
6.  **Model Artifacts:** Each trained model is saved as an MLflow artifact using `mlflow.sklearn.log_model()`, allowing for easy retrieval and deployment later.

By logging each model's parameters and performance metrics within separate runs under the same experiment, MLflow provides a clear interface to compare their effectiveness and identify the best performing model for the given task.

### How it Works

The code uses a `with mlflow.start_run(...)` block for each model. This ensures that all parameters, metrics, and the model logged within that block are associated with that specific run. The experiment name is set once at the beginning, grouping all these individual model runs together.

In [3]:
##Difernt models isis:

import mlflow
import mlflow.sklearn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target
target_names = iris.target_names # To map target indices to names

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# --- Set MLflow Experiment ---
# It's crucial to set the experiment name *before* starting any runs if you want them grouped.
mlflow.set_experiment("Iris Classification Models")

print("Starting Iris Classification Model Comparison...")

# --- Run 1: K-Nearest Neighbors (KNN) ---
with mlflow.start_run(run_name="KNN Classifier"):
    print("\n--- Training KNN Classifier ---")
    # Define hyperparameters
    n_neighbors = 5
    weights = "uniform"

    # Log parameters
    mlflow.log_param("model_type", "K-Nearest Neighbors")
    mlflow.log_param("n_neighbors", n_neighbors)
    mlflow.log_param("weights", weights)

    # Initialize and train the model
    knn_model = KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights)
    knn_model.fit(X_train, y_train)

    # Make predictions
    y_pred = knn_model.predict(X_test)

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    # For multiclass, precision, recall, f1_score need 'average' parameter
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    # Log metrics
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)

    # Log the model
    mlflow.sklearn.log_model(knn_model, "knn_model")

    print(f"  KNN Metrics: Accuracy={accuracy:.4f}, Precision={precision:.4f}, Recall={recall:.4f}, F1-Score={f1:.4f}")
    print(f"  KNN Model saved to: {mlflow.active_run().info.artifact_uri}/knn_model")


# --- Run 2: Support Vector Machine (SVM) ---
with mlflow.start_run(run_name="SVM Classifier"):
    print("\n--- Training SVM Classifier ---")
    # Define hyperparameters
    C = 1.0
    kernel = "rbf"
    gamma = "scale"

    # Log parameters
    mlflow.log_param("model_type", "Support Vector Machine")
    mlflow.log_param("C", C)
    mlflow.log_param("kernel", kernel)
    mlflow.log_param("gamma", gamma)

    # Initialize and train the model
    svm_model = SVC(C=C, kernel=kernel, gamma=gamma, random_state=42)
    svm_model.fit(X_train, y_train)

    # Make predictions
    y_pred = svm_model.predict(X_test)

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    # Log metrics
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)

    # Log the model
    mlflow.sklearn.log_model(svm_model, "svm_model")

    print(f"  SVM Metrics: Accuracy={accuracy:.4f}, Precision={precision:.4f}, Recall={recall:.4f}, F1-Score={f1:.4f}")
    print(f"  SVM Model saved to: {mlflow.active_run().info.artifact_uri}/svm_model")


# --- Run 3: Decision Tree Classifier ---
with mlflow.start_run(run_name="Decision Tree Classifier"):
    print("\n--- Training Decision Tree Classifier ---")
    # Define hyperparameters
    max_depth = 5
    min_samples_leaf = 5

    # Log parameters
    mlflow.log_param("model_type", "Decision Tree")
    mlflow.log_param("max_depth", max_depth)
    mlflow.log_param("min_samples_leaf", min_samples_leaf)

    # Initialize and train the model
    dt_model = DecisionTreeClassifier(max_depth=max_depth, min_samples_leaf=min_samples_leaf, random_state=42)
    dt_model.fit(X_train, y_train)

    # Make predictions
    y_pred = dt_model.predict(X_test)

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    # Log metrics
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)

    # Log the model
    mlflow.sklearn.log_model(dt_model, "decision_tree_model")

    print(f"  Decision Tree Metrics: Accuracy={accuracy:.4f}, Precision={precision:.4f}, Recall={recall:.4f}, F1-Score={f1:.4f}")
    print(f"  Decision Tree Model saved to: {mlflow.active_run().info.artifact_uri}/decision_tree_model")

print("\nAll Iris classification models trained and logged to MLflow.")
print("To view results, run 'mlflow ui' in your terminal and navigate to the 'Iris Classification Models' experiment.")

Starting Iris Classification Model Comparison...

--- Training KNN Classifier ---


2025/07/04 14:01:22 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/04 14:01:27 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/07/04 14:01:27 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


  KNN Metrics: Accuracy=1.0000, Precision=1.0000, Recall=1.0000, F1-Score=1.0000
  KNN Model saved to: file:///C:/Users/MiguelAngelSanchezRa/Python%20Sandbox/Corporate%20training/BA%20MLOps/02%20Building%20%26%20Integrating%20ML%20Pipelines/ML_Flow/mlruns/227057849671184419/689e3191300a469d881cde3077e7db59/artifacts/knn_model

--- Training SVM Classifier ---


2025/07/04 14:01:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/07/04 14:01:31 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


  SVM Metrics: Accuracy=1.0000, Precision=1.0000, Recall=1.0000, F1-Score=1.0000
  SVM Model saved to: file:///C:/Users/MiguelAngelSanchezRa/Python%20Sandbox/Corporate%20training/BA%20MLOps/02%20Building%20%26%20Integrating%20ML%20Pipelines/ML_Flow/mlruns/227057849671184419/1c13cd53b6754b70ba8c6f859360c591/artifacts/svm_model

--- Training Decision Tree Classifier ---


2025/07/04 14:01:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


  Decision Tree Metrics: Accuracy=1.0000, Precision=1.0000, Recall=1.0000, F1-Score=1.0000
  Decision Tree Model saved to: file:///C:/Users/MiguelAngelSanchezRa/Python%20Sandbox/Corporate%20training/BA%20MLOps/02%20Building%20%26%20Integrating%20ML%20Pipelines/ML_Flow/mlruns/227057849671184419/2cfad32c56d3403c8c44eeba2ebdf492/artifacts/decision_tree_model

All Iris classification models trained and logged to MLflow.
To view results, run 'mlflow ui' in your terminal and navigate to the 'Iris Classification Models' experiment.


In essence, this code runs three separate machine learning classification experiments on the same dataset (Iris) and uses MLflow to automatically record all the key details—the model's settings (parameters), its performance (metrics), and the trained model itself (artifacts)—for each experiment.

# MLflow: Preprocessing, Multiple Models, and Cross-Validation

## Robust Machine Learning Workflow for Titanic Survival Prediction

This example demonstrates a comprehensive machine learning workflow for the Kaggle Titanic survival prediction problem. It focuses on building robust, evaluable, and traceable models by incorporating data preprocessing, comparing multiple model types, and utilizing cross-validation, all tracked meticulously with MLflow.

### Purpose of This Example

The main objective is to showcase an end-to-end ML pipeline within the MLflow framework, covering:

* **Data Loading and Initial Cleaning:** Handling the input CSV files and dropping irrelevant columns like `PassengerId`, `Name`, `Ticket`, and `Cabin`.

* **Automated Preprocessing Pipelines:** Defining and applying robust preprocessing steps for both numerical and categorical features. This includes:
    * **Numerical Features (`Age`, `Fare`, `SibSp`, `Parch`):** Imputing missing values with the mean and scaling features using `StandardScaler`.
    * **Categorical Features (`Pclass`, `Sex`, `Embarked`):** Imputing missing values with the most frequent value and converting categorical data into numerical format using `OneHotEncoder`.
    * These steps are encapsulated within `sklearn.pipeline.Pipeline` and combined using `sklearn.compose.ColumnTransformer` for a streamlined process.

* **MLflow Experiment Setup:** Setting up a dedicated MLflow experiment (`Titanic CV & Model Logging`) to group all runs related to this project, ensuring organized tracking.

* **Cross-Validation for Robust Evaluation:** Employing `StratifiedKFold` cross-validation to assess model performance more reliably across different subsets of the training data. This helps in getting a more generalized estimate of the model's performance and reducing overfitting bias.

* **Multiple Model Comparison:** Training and evaluating three distinct classification algorithms:
    * **Logistic Regression:** A linear model for binary classification.
    * **Random Forest Classifier:** An ensemble tree-based method known for its robustness.
    * **Gradient Boosting Classifier:** Another powerful ensemble method that builds trees sequentially.

* **Comprehensive MLflow Logging:** For each model and cross-validation run, MLflow logs are captured:
    * **Parameters:** Model hyperparameters, data split configurations, and preprocessing details.
    * **Metrics:** Mean and standard deviation of cross-validation scores (accuracy, precision, recall, F1-score) to provide a statistical summary of performance.
    * **Models as Artifacts:** The entire `sklearn.pipeline.Pipeline` (including preprocessing and the final classifier) is logged as an MLflow artifact, making it easy to reproduce predictions or deploy the exact trained workflow.

This structured approach ensures that every aspect of the model development process is recorded, enabling easy comparison, reproducibility, and auditing of machine learning experiments.

### Data Requirements

This script expects `train.csv` and `test.csv` files from the Kaggle Titanic competition to be present in a `data/` subdirectory relative to where the script is run. If these files are not found, the script will exit and provide instructions for downloading them.

### How to See Your Results: The MLflow UI

After running this script, you can visualize and compare all the logged experiments and runs using the MLflow Tracking UI:

-   Open your terminal in the directory where your 'mlruns' folder is located (this is usually the directory from which you ran the Python script).
-   Execute the command:
    -   `mlflow ui`
-   This will launch the MLflow Tracking UI, typically accessible in your web browser at `http://localhost:5000`.
-   You'll be able to navigate to the `Titanic CV & Model Logging` experiment and inspect the details of each model's run, including logged parameters, metrics (mean and std of CV scores), and the saved `final_model_pipeline` artifact.

In [ ]:
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import numpy as np
import os

# --- Configuration ---
N_SPLITS_CV = 5 # Number of folds for cross-validation

# --- 1. Load Data ---
# You need train.csv and test.csv from Kaggle Titanic competition
TRAIN_FILE = "data/train.csv"
TEST_FILE = "data/test.csv"

if not os.path.exists(TRAIN_FILE) or not os.path.exists(TEST_FILE):
    print(f"Required files ({TRAIN_FILE}, {TEST_FILE}) not found.")
    print("Please download them from Kaggle Titanic competition:")
    print("https://www.kaggle.com/c/titanic/data")
    exit()

df_train = pd.read_csv(TRAIN_FILE)
df_test = pd.read_csv(TEST_FILE) # This is the "unseen" Kaggle test set

# Store PassengerId for potential use, though not used for submission anymore
test_passenger_ids = df_test['PassengerId']

# Drop 'PassengerId', 'Name', 'Ticket', 'Cabin' from both train and test data for this example
cols_to_drop = ['PassengerId', 'Name', 'Ticket', 'Cabin']
df_train_processed = df_train.drop(columns=[col for col in cols_to_drop if col in df_train.columns])
df_test_processed = df_test.drop(columns=[col for col in cols_to_drop if col in df_test.columns])


# Define target and features
target = 'Survived'
X_full = df_train_processed.drop(target, axis=1)
y_full = df_train_processed[target]

# Define feature types for preprocessing
numerical_features = ['Age', 'Fare', 'SibSp', 'Parch']
categorical_features = ['Pclass', 'Sex', 'Embarked']

# --- 2. Define Preprocessing Pipeline (with Cleaning Steps) ---

# Preprocessing for numerical features
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Preprocessing for categorical features
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Create a preprocessor using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='drop'
)

# --- Set MLflow Experiment ---
mlflow.set_experiment("Titanic CV & Model Logging") # Renamed experiment slightly
print("Starting Titanic Classification with CV and Model Logging...")

# Helper function to run CV, evaluate, and log to MLflow
def train_evaluate_log_model(model_name, classifier, X_full, y_full, X_test_kaggle, test_passenger_ids, model_params):
    with mlflow.start_run(run_name=f"{model_name}_CV_Run"): # Renamed run name slightly
        print(f"\n--- Training {model_name} with Cross-Validation ---")

        # Log parameters specific to this run
        mlflow.log_param("model_type", model_name)
        mlflow.log_param("n_splits_cv", N_SPLITS_CV)
        mlflow.log_param("cleaning_steps", "Age Impute Mean, Embarked Impute Mode")
        mlflow.log_param("preprocessing_steps", "StandardScaler, OneHotEncoder")
        mlflow.log_param("numerical_features", str(numerical_features))
        mlflow.log_param("categorical_features", str(categorical_features))
        mlflow.log_params(model_params)

        # Create the full pipeline including preprocessing and classifier
        full_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                        ('classifier', classifier)])

        # --- Perform Cross-Validation ---
        cv = StratifiedKFold(n_splits=N_SPLITS_CV, shuffle=True, random_state=42)
        scoring = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted']
        cv_results = cross_validate(full_pipeline, X_full, y_full, cv=cv, scoring=scoring, return_train_score=False)

        # Log mean and standard deviation of CV metrics
        print(f"  {model_name} CV Results:")
        for metric in scoring:
            mean_metric = np.mean(cv_results[f'test_{metric}'])
            std_metric = np.std(cv_results[f'test_{metric}'])
            mlflow.log_metric(f"cv_mean_{metric}", mean_metric)
            mlflow.log_metric(f"cv_std_{metric}", std_metric)
            print(f"    Mean Test {metric}: {mean_metric:.4f} (Std: {std_metric:.4f})")

        # --- Train the final model on the full training data ---
        # This model will be saved to MLflow artifacts
        print(f"  Fitting final {model_name} pipeline on full training data for logging...")
        full_pipeline.fit(X_full, y_full)

        # Log the final trained pipeline
        mlflow.sklearn.log_model(full_pipeline, "final_model_pipeline")
        print(f"  Final {model_name} pipeline logged to: {mlflow.active_run().info.artifact_uri}/final_model_pipeline")

        # Note: Predictions on X_test_kaggle and submission file saving are removed.


# --- Define Models and their Parameters ---
models_to_compare = [
    {
        "name": "Logistic Regression",
        "classifier": LogisticRegression(C=0.1, solver='liblinear', max_iter=1000, random_state=42),
        "params": {"C": 0.1, "solver": "liblinear", "max_iter": 1000}
    },
    {
        "name": "Random Forest Classifier",
        "classifier": RandomForestClassifier(n_estimators=100, max_depth=8, random_state=42),
        "params": {"n_estimators": 100, "max_depth": 8}
    },
    {
        "name": "Gradient Boosting Classifier",
        "classifier": GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42),
        "params": {"n_estimators": 100, "learning_rate": 0.1, "max_depth": 3}
    }
]

# --- Run the full pipeline for each model ---
for model_info in models_to_compare:
    train_evaluate_log_model(
        model_info["name"],
        model_info["classifier"],
        X_full,
        y_full,
        df_test_processed, # X_test_kaggle is still passed, but not used for predictions in this version
        test_passenger_ids, # test_passenger_ids is still passed, but not used in this version
        model_info["params"]
    )

print("\nAll Titanic models processed with CV and models logged to MLflow.")
print("To view results, run 'mlflow ui' in your terminal and navigate to the 'Titanic CV & Model Logging' experiment.")


In essence, this code sets up a robust machine learning workflow for the Titanic dataset that automatically handles data preparation and evaluates multiple models using cross-validation, while meticulously logging all parameters, cross-validation metrics, and the final trained models to MLflow for easy comparison and reproducibility.


After some refractoring: 

In [4]:
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import numpy as np
import os

# --- Configuration (Constants defined globally or passed as arguments) ---
N_SPLITS_CV = 5 # Number of folds for cross-validation
TRAIN_FILE = "data/train.csv"
TEST_FILE = "data/test.csv"

# --- Helper function to run CV, evaluate, and log to MLflow ---
# This function encapsulates the logic for a single model's training and logging
# It now correctly accepts 'preprocessor' and 'N_SPLITS_CV' as arguments
def train_evaluate_log_model(model_name, classifier, X_full, y_full, model_params, preprocessor, n_splits_cv_param):
    """
    Trains, evaluates (with cross-validation), and logs a machine learning model to MLflow.

    Args:
        model_name (str): A descriptive name for the model (e.g., "Logistic Regression").
        classifier (sklearn estimator): The scikit-learn classifier object (e.g., LogisticRegression()).
        X_full (pd.DataFrame): The full feature dataset for training.
        y_full (pd.Series): The full target dataset for training.
        model_params (dict): Dictionary of hyperparameters for the classifier.
        preprocessor (sklearn.compose.ColumnTransformer): The defined data preprocessing pipeline.
        n_splits_cv_param (int): Number of folds for cross-validation.
    """
    with mlflow.start_run(run_name=f"{model_name}_CV_Run"):
        print(f"\n--- Training {model_name} with Cross-Validation ---")

        # Log parameters specific to this run
        mlflow.log_param("model_type", model_name)
        mlflow.log_param("n_splits_cv", n_splits_cv_param)
        mlflow.log_param("cleaning_steps", "Age Impute Mean, Embarked Impute Mode")
        mlflow.log_param("preprocessing_steps", "StandardScaler, OneHotEncoder")
        # Log lists as strings for simplicity in MLflow UI param table
        mlflow.log_param("numerical_features", str(['Age', 'Fare', 'SibSp', 'Parch']))
        mlflow.log_param("categorical_features", str(['Pclass', 'Sex', 'Embarked']))
        mlflow.log_params(model_params) # Log the specific model hyperparameters

        # Create the full pipeline including preprocessing and classifier
        full_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                         ('classifier', classifier)])

        # --- Perform Cross-Validation ---
        cv = StratifiedKFold(n_splits=n_splits_cv_param, shuffle=True, random_state=42)
        scoring = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted']
        cv_results = cross_validate(full_pipeline, X_full, y_full, cv=cv, scoring=scoring, return_train_score=False)

        # Log mean and standard deviation of CV metrics
        print(f"  {model_name} CV Results:")
        for metric in scoring:
            mean_metric = np.mean(cv_results[f'test_{metric}'])
            std_metric = np.std(cv_results[f'test_{metric}'])
            mlflow.log_metric(f"cv_mean_{metric}", mean_metric)
            mlflow.log_metric(f"cv_std_{metric}", std_metric)
            print(f"    Mean Test {metric}: {mean_metric:.4f} (Std: {std_metric:.4f})")

        # --- Train the final model on the full training data ---
        # This model will be saved to MLflow artifacts for potential deployment
        print(f"  Fitting final {model_name} pipeline on full training data for logging...")
        full_pipeline.fit(X_full, y_full)

        # Log the final trained pipeline
        mlflow.sklearn.log_model(full_pipeline, "final_model_pipeline")
        print(f"  Final {model_name} pipeline logged to: {mlflow.active_run().info.artifact_uri}/final_model_pipeline")


# --- Main Execution Function ---
def main():
    # --- 1. Load Data ---
    # Check if data files exist
    if not os.path.exists("data"):
        os.makedirs("data") # Create data directory if it doesn't exist
    if not os.path.exists(TRAIN_FILE) or not os.path.exists(TEST_FILE):
        print(f"Required files ({TRAIN_FILE}, {TEST_FILE}) not found.")
        print("Please download them from Kaggle Titanic competition:")
        print("https://www.kaggle.com/c/titanic/data")
        print("Place 'train.csv' and 'test.csv' inside the 'data/' directory.")
        return # Exit the main function if files are missing

    df_train = pd.read_csv(TRAIN_FILE)
    df_test = pd.read_csv(TEST_FILE) # This is the "unseen" Kaggle test set

    # Drop 'PassengerId', 'Name', 'Ticket', 'Cabin' from both train and test data for this example
    cols_to_drop = ['PassengerId', 'Name', 'Ticket', 'Cabin']
    df_train_processed = df_train.drop(columns=[col for col in cols_to_drop if col in df_train.columns])
    df_test_processed = df_test.drop(columns=[col for col in cols_to_drop if col in df_test.columns]) # Keep this for consistent column dropping, even if not used for prediction in this example

    # Define target and features
    target = 'Survived'
    X_full = df_train_processed.drop(target, axis=1)
    y_full = df_train_processed[target]

    # Define feature types for preprocessing
    numerical_features = ['Age', 'Fare', 'SibSp', 'Parch']
    categorical_features = ['Pclass', 'Sex', 'Embarked']

    # --- 2. Define Preprocessing Pipeline (with Cleaning Steps) ---
    # Preprocessing for numerical features
    numerical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])

    # Preprocessing for categorical features
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    # Create a preprocessor using ColumnTransformer
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_features),
            ('cat', categorical_transformer, categorical_features)
        ],
        remainder='drop' # Drops any columns not specified in numerical_features or categorical_features
    )

    # --- Set MLflow Experiment ---
    mlflow.set_experiment("Titanic CV & Model Logging")
    print("Starting Titanic Classification with CV and Model Logging...")

    # --- Define Models and their Parameters ---
    models_to_compare = [
        {
            "name": "Logistic Regression",
            "classifier": LogisticRegression(C=0.1, solver='liblinear', max_iter=1000, random_state=42),
            "params": {"C": 0.1, "solver": "liblinear", "max_iter": 1000}
        },
        {
            "name": "Random Forest Classifier",
            "classifier": RandomForestClassifier(n_estimators=100, max_depth=8, random_state=42),
            "params": {"n_estimators": 100, "max_depth": 8}
        },
        {
            "name": "Gradient Boosting Classifier",
            "classifier": GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42),
            "params": {"n_estimators": 100, "learning_rate": 0.1, "max_depth": 3}
        }
    ]

    # --- Run the full pipeline for each model ---
    for model_info in models_to_compare:
        train_evaluate_log_model(
            model_info["name"],
            model_info["classifier"],
            X_full,
            y_full,
            model_info["params"],
            preprocessor, # Pass the preprocessor defined in main()
            N_SPLITS_CV   # Pass the N_SPLITS_CV constant
        )

    print("\nAll Titanic models processed with CV and models logged to MLflow.")
    print("To view results, run 'mlflow ui' in your terminal and navigate to the 'Titanic CV & Model Logging' experiment.")

# --- Guard to run main() when script is executed directly ---
if __name__ == "__main__":
    main()

Starting Titanic Classification with CV and Model Logging...

--- Training Logistic Regression with Cross-Validation ---
  Logistic Regression CV Results:
    Mean Test accuracy: 0.8014 (Std: 0.0134)


2025/07/04 14:25:18 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


    Mean Test precision_weighted: 0.8002 (Std: 0.0136)
    Mean Test recall_weighted: 0.8014 (Std: 0.0134)
    Mean Test f1_weighted: 0.7988 (Std: 0.0135)
  Fitting final Logistic Regression pipeline on full training data for logging...


2025/07/04 14:25:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


  Final Logistic Regression pipeline logged to: file:///C:/Users/MiguelAngelSanchezRa/Python%20Sandbox/Corporate%20training/BA%20MLOps/02%20Building%20%26%20Integrating%20ML%20Pipelines/ML_Flow/mlruns/142701399195007350/680e1ef1e0124ac0839a5a92684bb404/artifacts/final_model_pipeline

--- Training Random Forest Classifier with Cross-Validation ---
  Random Forest Classifier CV Results:
    Mean Test accuracy: 0.8406 (Std: 0.0170)
    Mean Test precision_weighted: 0.8418 (Std: 0.0179)
    Mean Test recall_weighted: 0.8406 (Std: 0.0170)


...

  Final Random Forest Classifier pipeline logged to: file:///C:/Users/MiguelAngelSanchezRa/Python%20Sandbox/Corporate%20training/BA%20MLOps/02%20Building%20%26%20Integrating%20ML%20Pipelines/ML_Flow/mlruns/142701399195007350/aad05072b85e4de483d9742f928ae935/artifacts/final_model_pipeline

--- Training Gradient Boosting Classifier with Cross-Validation ---
  Gradient Boosting Classifier CV Results:
    Mean Test accuracy: 0.8227 (Std: 0.0152)
    Mean Test precision_weighted: 0.8235 (Std: 0.0145)
    Mean Test recall_weighted: 0.8227 (Std: 0.0152)


2025/07/04 14:25:31 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


    Mean Test f1_weighted: 0.8191 (Std: 0.0167)
  Fitting final Gradient Boosting Classifier pipeline on full training data for logging...


2025/07/04 14:25:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


  Final Gradient Boosting Classifier pipeline logged to: file:///C:/Users/MiguelAngelSanchezRa/Python%20Sandbox/Corporate%20training/BA%20MLOps/02%20Building%20%26%20Integrating%20ML%20Pipelines/ML_Flow/mlruns/142701399195007350/3e09e630631343638cb3f5a099e088d7/artifacts/final_model_pipeline

All Titanic models processed with CV and models logged to MLflow.
To view results, run 'mlflow ui' in your terminal and navigate to the 'Titanic CV & Model Logging' experiment.


# MLflow: Calling a Logged Model - Single Data Point

## Demonstrating Model Loading and Prediction for a Single Entry

This section illustrates how to load a previously logged MLflow model and use it to make a prediction for a single new, unseen data point. This is a fundamental step in the model deployment and inference phase of the machine learning lifecycle, often used for real time predictions or individual queries.

### Purpose of This Example

The primary objective is to demonstrate the ease with which MLflow allows you to retrieve and utilize a trained model artifact. Specifically, it covers:

1.  **Identifying the Model:** How to specify the exact model to load using its MLflow Run ID and the artifact path under which it was saved.
2.  **Constructing the Model URI:** Forming the unique MLflow URI (`runs:/<RUN_ID>/<ARTIFACT_PATH>`) that points directly to the logged model.
3.  **Loading the Model:** Using `mlflow.sklearn.load_model()` (or the appropriate flavor-specific `load_model` function) to deserialize the model pipeline into memory.
4.  **Making a Single Prediction:** Applying the loaded model to a single new input data row to generate a prediction.
5.  **Error Handling:** Including a `try-except` block to gracefully handle scenarios where the model cannot be loaded, providing helpful debugging information.

This process is fundamental for integrating MLflow managed models into production systems, especially for scenarios requiring individual predictions.

### Key Considerations

* **`MLFLOW_RUN_ID`:** This is the unique identifier for the specific MLflow run during which your desired model was trained and logged. You can find this ID in the MLflow UI or from the output of the training script (e.g., from the `mlflow.active_run().info.run_id` printout).
* **`MODEL_ARTIFACT_PATH`:** This is the name you gave to the model when it was logged (e.g., `"final_model_pipeline"` or `"decision_tree_classifier_model"`). It's the sub-path within the run's artifact directory.
* **Input Data Format:** The new data (`new_passenger` in this example) used for prediction **must** have the same feature columns and data types as the data used to train the original model. If the original model included a preprocessing pipeline (as in the Titanic example), the loaded object will be the entire pipeline, and you should pass raw, un-preprocessed data to it as a Pandas DataFrame, even for a single row.
* **Environment:** Ensure that the Python environment where you are loading the model has all the necessary libraries installed (e.g., `scikit-learn`, `pandas`, `mlflow`) that were used when the model was originally saved.

This example uses a hardcoded `MLFLOW_RUN_ID` for demonstration purposes. In a real world scenario, this ID might be dynamically retrieved from a model registry, a database, or a configuration file.

In [5]:
import mlflow
import mlflow.sklearn
import pandas as pd # Import pandas

# --- Configuration ---
# IMPORTANT: Provide the specific Run ID you want to test here.
# This should be the ID of a run where your 'final_model_pipeline' was logged.
MLFLOW_RUN_ID = "a3a7aff216464fefab633d2962aed98d" # <--- Your specified Run ID

# Path to the model artifact within the run
# This should match the name given when the model was logged (e.g., "final_model_pipeline")
MODEL_ARTIFACT_PATH = "final_model_pipeline"

# Construct the MLflow model URI
MODEL_URI = f"runs:/{MLFLOW_RUN_ID}/{MODEL_ARTIFACT_PATH}"

try:
    # Load the model directly using the specified MLFLOW_RUN_ID and MODEL_ARTIFACT_PATH
    loaded_pipeline = mlflow.sklearn.load_model(MODEL_URI)

    print(f"\n--- Demonstrating loaded model prediction from Run ID: {MLFLOW_RUN_ID} ---")
    print(f"Loaded pipeline from MLflow URI: {MODEL_URI}")

    # Create new unseen data (MUST have the same columns as original X)
    # Ensure column names match the features used during training and preprocessing
    new_passenger = pd.DataFrame([{
        'Pclass': 3, 'Sex': 'male', 'Age': 28.0, 'SibSp': 0,
        'Parch': 0, 'Fare': 10.0, 'Embarked': 'S'
    }])

    predicted_survival = loaded_pipeline.predict(new_passenger)[0]
    print(f"New passenger data:\n{new_passenger}")
    print(f"Predicted Survival (0=No, 1=Yes): {predicted_survival}")

except Exception as e:
    print(f"\nCould not demonstrate model loading: {e}")
    print(f"Error details: {e}")
    print("Please ensure:")
    print(f"1. The MLflow run with ID '{MLFLOW_RUN_ID}' exists.")
    print(f"2. The model artifact '{MODEL_ARTIFACT_PATH}' exists within that run's artifacts.")
    print("3. Your 'mlruns' directory is accessible from where you are running this script.")


--- Demonstrating loaded model prediction from Run ID: a3a7aff216464fefab633d2962aed98d ---
Loaded pipeline from MLflow URI: runs:/a3a7aff216464fefab633d2962aed98d/final_model_pipeline
New passenger data:
   Pclass   Sex   Age  SibSp  Parch   Fare Embarked
0       3  male  28.0      0      0   10.0        S
Predicted Survival (0=No, 1=Yes): 0


# MLflow: Calling a Logged Model - Multiple Data Points

## Demonstrating Model Loading and Prediction for Multiple Entries

This section extends the previous example to show how to load an MLflow model and use it to make predictions for an array or a batch of new, unseen data points. This is a common scenario for batch processing or when serving multiple requests simultaneously.

### Purpose of This Example

The primary objective is to demonstrate how to efficiently use a loaded MLflow model to predict outcomes for multiple inputs. Specifically, it covers:

1.  **Re-loading the Model:** The process of loading the model remains the same, emphasizing that the loaded object can handle multiple rows of input.
2.  **Preparing Batch Input:** Creating a Pandas DataFrame containing several new data points, ensuring that the structure (column names and types) matches the expectations of the trained model/pipeline.
3.  **Batch Prediction:** Applying the `predict()` method of the loaded model to the entire DataFrame of new data, which returns an array of predictions.
4.  **Integrating Predictions:** Demonstrating how to add the predictions back into the input DataFrame for easy interpretation and analysis.

This capability is essential for scalable inference and integrating MLflow models into data pipelines or applications that process multiple records at once.

### Key Considerations

* **`MLFLOW_RUN_ID` and `MODEL_ARTIFACT_PATH`:** As before, these must accurately point to the desired logged model.
* **Batch Input Format:** When providing multiple data points, they should be structured as rows in a Pandas DataFrame. Each row represents a single instance for which a prediction is desired, and all columns must correspond to the features the model was trained on.
* **Output:** The `predict()` method will return an an array of predictions, where each element corresponds to the prediction for the respective row in the input DataFrame.

In [ ]:
import mlflow
import mlflow.sklearn
import pandas as pd # Import pandas

# --- Configuration for Model Loading ---
# IMPORTANT: Replace this with the actual Run ID from your MLflow experiment.
# This should be the ID of a run where your 'final_model_pipeline' was logged.
MLFLOW_RUN_ID = "a3a7aff216464fefab633d2962aed98d" # <--- Your specified Run ID goes here

# Path to the model artifact within the specified run's artifacts.
# This should match the name given when the model was logged (e.g., "final_model_pipeline").
MODEL_ARTIFACT_PATH = "final_model_pipeline"

# Construct the full MLflow model URI. This URI uniquely identifies the model artifact.
MODEL_URI = f"runs:/{MLFLOW_RUN_ID}/{MODEL_ARTIFACT_PATH}"

try:
    # Load the model directly using the specified MLflow URI.
    # MLflow automatically handles downloading artifacts and reconstructing the model.
    loaded_pipeline = mlflow.sklearn.load_model(MODEL_URI)

    print(f"\n--- Demonstrating loaded model prediction from Run ID: {MLFLOW_RUN_ID} ---")
    print(f"Loaded pipeline from MLflow URI: {MODEL_URI}")

    # Create new unseen data for prediction. This is a DataFrame with multiple passengers.
    # Ensure column names and order match the features used during training and preprocessing.
    new_passengers_data = [
        # Passenger 1 (Example: likely to survive)
        {'Pclass': 1, 'Sex': 'female', 'Age': 25.0, 'SibSp': 1, 'Parch': 0, 'Fare': 75.0, 'Embarked': 'C'},
        # Passenger 2 (Example: likely not to survive)
        {'Pclass': 3, 'Sex': 'male', 'Age': 35.0, 'SibSp': 0, 'Parch': 0, 'Fare': 8.0, 'Embarked': 'S'},
        # Passenger 3 (Example: another likely to survive)
        {'Pclass': 2, 'Sex': 'female', 'Age': 40.0, 'SibSp': 2, 'Parch': 1, 'Fare': 45.0, 'Embarked': 'Q'}
    ]
    new_passengers_df = pd.DataFrame(new_passengers_data)

    print(f"New passenger data:\n{new_passengers_df}")

    # Make predictions for all passengers in the DataFrame.
    predicted_survivals = loaded_pipeline.predict(new_passengers_df)

    print(f"Predicted Survival for all passengers (0=No, 1=Yes):\n{predicted_survivals}")

    # Optionally, add the predictions back to the DataFrame for easy viewing.
    new_passengers_df['Predicted_Survival'] = predicted_survivals
    print("\nDataFrame with Predictions:")
    print(new_passengers_df)

except Exception as e:
    # Catch any exceptions during model loading or prediction and provide informative messages.
    print(f"\nCould not demonstrate model loading: {e}")
    print(f"Error details: {e}")
    print("Please ensure the following:")
    print(f"1. The MLflow run with ID '{MLFLOW_RUN_ID}' exists and is accessible.")
    print(f"2. The model artifact '{MODEL_ARTIFACT_PATH}' exists within that run's artifacts directory.")
    print("3. Your 'mlruns' directory (or configured MLflow tracking URI) is accessible from where you are running this script.")
    print("4. All necessary Python libraries (e.g., scikit-learn, pandas) are installed in your environment.")
    print("5. The input data for prediction has the exact same column names and structure as the training data.")


--- Demonstrating loaded model prediction from Run ID: a3a7aff216464fefab633d2962aed98d ---
Loaded pipeline from MLflow URI: runs:/a3a7aff216464fefab633d2962aed98d/final_model_pipeline
New passenger data:
   Pclass     Sex   Age  SibSp  Parch   Fare Embarked
0       1  female  25.0      1      0   75.0        C
1       3    male  35.0      0      0    8.0        S
2       2  female  40.0      2      1   45.0        Q
Predicted Surval for all passengers (0=No, 1=Yes):
[1 0 1]

DataFrame with Predictions:
   Pclass     Sex   Age  SibSp  Parch   Fare Embarked  Predicted_Survival
0       1  female  25.0      1      0   75.0        C                   1
1       3    male  35.0      0      0    8.0        S                   0
2       2  female  40.0      2      1   45.0        Q                   1


# MLflow: FastAPI Deployment

## Deploying a Logged MLflow Model as a Web API with FastAPI

This section demonstrates how to deploy a machine learning model, previously logged with MLflow, as a RESTful API using FastAPI. FastAPI is a modern, fast (high-performance) web framework for building APIs with Python 3.7+ based on standard Python type hints.

### Purpose of This Example

The primary goal is to create a simple, runnable web service that can receive new data and return predictions using the MLflow-logged model. This involves:

1.  **FastAPI Application Setup:** Initializing a FastAPI application.
2.  **Model Loading on Startup:** Implementing an `@app.on_event("startup")` function to load the MLflow model only once when the API starts. This prevents redundant model loading for every request, improving performance.
3.  **Input Data Validation with Pydantic:** Defining a `Pydantic` `BaseModel` (`TitanicPassenger`) to strictly validate the structure and types of incoming request data. This ensures that the input data conforms to the features expected by our trained model.
4.  **Prediction Endpoint:** Creating a POST endpoint (`/predict`) that accepts passenger data, converts it into a Pandas DataFrame (the format expected by our `scikit-learn` pipeline), uses the loaded MLflow model to make a prediction, and returns the result.
5.  **Robust Error Handling:** Including `try-except` blocks to catch potential issues during model loading or prediction, providing informative HTTP error responses.

This setup is ideal for integrating your MLflow models into larger applications, microservices architectures, or providing real-time inference capabilities.

### Key Components

* **`mlflow`:** Used for loading the pre-trained model from its MLflow URI.
* **`fastapi`:** The web framework for building the API.
* **`pydantic`:** Used by FastAPI for data validation and serialization/deserialization.
* **`uvicorn`:** An ASGI server that runs the FastAPI application.
* **`pandas`:** For structuring input data into a DataFrame format expected by the `scikit-learn` pipeline.

**Before Running:**

Make sure you have `mlflow`, `fastapi`, `uvicorn`, and `pandas` installed:
```bash
pip install mlflow fastapi uvicorn pandas
```

**Important Note on `MLFLOW_RUN_ID`:**

You **must** replace the placeholder `MLFLOW_RUN_ID` with the actual Run ID of one of your trained models from the previous MLflow experiments (e.g., from the Titanic CV & Model Logging experiment). This ID tells FastAPI which specific model to load from your MLflow tracking server or local `mlruns` directory.

In [ ]:
# app.py - This file defines the FastAPI application for model deployment

import mlflow
import pandas as pd
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import uvicorn
import os

# --- Configuration ---
# IMPORTANT: Replace with the actual Run ID of your BEST model from MLflow UI!
# This ID links to the specific run where your 'final_model_pipeline' was logged.
# Example: MLFLOW_RUN_ID = "a1b2c3d4e5f67890abcd1234"
MLFLOW_RUN_ID = "a3a7aff216464fefab633d2962aed98d" # <--- REPLACE THIS LINE WITH YOUR RUN ID!

# Path to the model artifact within the run's artifacts directory.
# This should match the name given when the model was logged (e.g., "final_model_pipeline").
MODEL_ARTIFACT_PATH = "final_model_pipeline"

# Construct the MLflow model URI. This URI tells MLflow where to find the model.
# If using a remote MLflow Tracking Server, you might need to set its URI first:
# mlflow.set_tracking_uri("http://your_mlflow_server_ip:5000")
MODEL_URI = f"runs:/{MLFLOW_RUN_ID}/{MODEL_ARTIFACT_PATH}"

# Initialize FastAPI app instance.
app = FastAPI(
    title="Titanic Survival Prediction API",
    description="API for predicting survival on the Titanic using an MLflow-logged model.",
    version="1.0.0"
)

# Global variable to hold the loaded model pipeline.
# It's initialized to None and loaded once on application startup.
model_pipeline = None

@app.on_event("startup")
async def load_model():
    """
    Load the MLflow model pipeline when the FastAPI application starts up.
    This is an asynchronous function that runs once at the beginning.
    It ensures the model is loaded only once and is available for all incoming requests,
    avoiding redundant loading and improving API performance.
    If the model fails to load, it raises an HTTPException, preventing the server from starting.
    """
    global model_pipeline # Declare that we are modifying the global 'model_pipeline' variable
    try:
        print(f"Loading model from MLflow URI: {MODEL_URI}")
        # Use mlflow.sklearn.load_model because the model was logged using mlflow.sklearn flavor.
        model_pipeline = mlflow.sklearn.load_model(MODEL_URI)
        print("Model loaded successfully!")
    except Exception as e:
        print(f"Error loading model: {e}")
        # Raise an HTTP exception to indicate a critical startup failure.
        raise HTTPException(status_code=500, detail=f"Model failed to load: {e}. "
                                                      f"Please check MLFLOW_RUN_ID and ensure MLflow tracking server "
                                                      f"is running correctly for local runs or configured for remote.")

# Define the input data schema using Pydantic.
# This ensures that incoming JSON requests conform to the expected structure and data types.
class TitanicPassenger(BaseModel):
    Pclass: int
    Sex: str
    Age: float
    SibSp: int
    Parch: int
    Fare: float
    Embarked: str

    # Example schema for OpenAPI documentation (Swagger UI).
    class Config:
        schema_extra = {
            "example": {
                "Pclass": 3,
                "Sex": "male",
                "Age": 28.0,
                "SibSp": 0,
                "Parch": 0,
                "Fare": 10.0,
                "Embarked": "S"
            }
        }

# Define the prediction endpoint.
@app.post("/predict")
async def predict_survival(passenger: TitanicPassenger):
    """
    Predicts survival for a single Titanic passenger based on the provided data.
    Expects a JSON body conforming to the TitanicPassenger schema.
    Returns the predicted survival status (0 or 1) and a human-readable string.
    """
    # Check if the model has been successfully loaded. If not, return a 503 Service Unavailable error.
    if model_pipeline is None:
        raise HTTPException(status_code=503, detail="Model not loaded yet. Please wait for startup or check logs.")

    # Convert the incoming Pydantic model data into a Pandas DataFrame.
    # This is crucial because scikit-learn pipelines typically expect DataFrame input.
    # .model_dump() is used for Pydantic v2; for v1, it would be .dict()
    input_df = pd.DataFrame([passenger.model_dump()]) 

    try:
        # Make a prediction using the loaded MLflow pipeline.
        # The pipeline handles all preprocessing steps internally.
        prediction = model_pipeline.predict(input_df)[0]
        survival_status = "Survived" if prediction == 1 else "Not Survived"
        
        # Return the prediction and a descriptive status.
        return {"prediction": int(prediction), "survival_status": survival_status}
    except Exception as e:
        # Catch any errors during the prediction process and return a 500 Internal Server Error.
        raise HTTPException(status_code=500, detail=f"Prediction failed: {e}. "
                                                      f"Ensure input data matches training features and types.")

# You can add a root endpoint for basic health check or info
@app.get("/", tags=["Health Check"])
async def root():
    return {"message": "Titanic Survival Prediction API is running. Visit /docs for API documentation."}


### Run the FastAPI Application

To run this FastAPI application:

1.  **Save the code:** Save the Python code from the previous cell into a file named `app.py` in your working directory.
2.  **Open your terminal:** Navigate to the directory where you saved `app.py`.
3.  **Execute the command:** Run the following command:
    ```bash
    uvicorn app:app --reload
    ```
    * `app:app` refers to the `app` object within the `app.py` file.
    * `--reload` enables auto-reloading, which is useful during development as it restarts the server whenever code changes are detected.

You should see output indicating that Uvicorn is running the server, typically on `http://127.0.0.1:8000` or `http://localhost:8000`.

### Test Your API

Once the server is running, you can test your API using a web browser or a tool like `curl` or Postman:

-   **Root Endpoint:** Open your web browser and go to: `http://localhost:8000/`
    * You should see a simple JSON response: `{"message": "Titanic Survival Prediction API is running. Visit /docs for API documentation."}`

-   **Interactive API Documentation (Swagger UI):** Open your web browser and go to: `http://localhost:8000/docs`
    * FastAPI automatically generates interactive API documentation (Swagger UI). You can see your `/predict` endpoint, the expected input schema, and example requests. You can use the "Try it out" button to send sample requests directly from your browser and see the responses.

-   **Alternative Documentation (ReDoc):** For a different documentation style, visit: `http://localhost:8000/redoc`

Once the model has been selected and deployed with FastAPI, you can then access it via web clients or programmatically via HTTP requests.

### Calling the Deployed MLflow Model with Python `requests`

This section demonstrates how to programmatically interact with the FastAPI model deployment using Python's `requests` library. This is how other applications or services would typically consume your deployed machine learning model.

### Purpose of This Example

The goal is to show how to send a JSON payload to the `/predict` endpoint of your running FastAPI application and process the JSON response. This includes:

1.  **Defining the API Endpoint:** Specifying the URL of your FastAPI prediction endpoint.
2.  **Preparing Request Headers:** Setting the `Content-Type` header to `application/json` to indicate that the request body is in JSON format.
3.  **Creating the Request Body:** Constructing a Python dictionary that matches the `TitanicPassenger` Pydantic schema defined in your `app.py`.
4.  **Sending the POST Request:** Using `requests.post()` to send the data to the API.
5.  **Handling the Response:** Checking the HTTP status code and parsing the JSON response from the API.

This client-side code simulates how a front-end application, another microservice, or a batch processing script would interact with your deployed model.

In [7]:
import requests # Import the requests library for making HTTP requests

# --- Configuration for API Call ---
url = "http://localhost:8000/predict" # The URL of your running FastAPI prediction endpoint
headers = {"Content-Type": "application/json"} # Specify that we are sending JSON data

# The data payload for a single passenger, matching the TitanicPassenger Pydantic model.
data = {
    "Pclass": 3,
    "Sex": "male",
    "Age": 28.0,
    "SibSp": 0,
    "Parch": 0,
    "Fare": 10.0,
    "Embarked": "S"
}

# Send the POST request to the FastAPI endpoint.
# The 'json' parameter automatically serializes the Python dict to JSON and sets the Content-Type header.
response = requests.post(url, headers=headers, json=data)

# Check the HTTP status code of the response.
if response.status_code == 200:
    print("Prediction successful:")
    # Parse the JSON response body and print it.
    print(response.json())
else:
    # If the request was not successful, print the status code and the error message.
    print(f"Error: {response.status_code} - {response.text}")
    print("Please ensure the FastAPI application (app.py) is running and accessible at http://localhost:8000.")


Prediction successful:
{'prediction': 0, 'survival_status': 'Not Survived'}


## Exercise

Replicate the Churn project using MLFlow. Test different algorithms like Logistic Regression, Random forest and Gradient Boost Classifier, select the best one and deploy it with MLFlow directly and also with FastAPI. Take the Titanic example as a baseline to build your solution. 

In [ ]:
##Answer:

import os
import mlflow
import mlflow.sklearn
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, train_test_split, cross_validate
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from mlflow.models import infer_signature
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import Schema, ColSpec

# --- Constants ---
N_SPLITS_CV = 5
RANDOM_SEED = 42
MLFLOW_EXPERIMENT_NAME = "Telco Churn Prediction CV & Model Logging"

# --- Data Loading and Initial Preprocessing ---
def load_data(data_path: str):
    """
    Loads the Telco Customer Churn dataset, performs initial cleaning,
    and separates features (X) from the target (y).
    """
    churn = pd.read_csv(data_path)
    churn['TotalCharges'] = pd.to_numeric(churn['TotalCharges'], errors='coerce')
    churn.dropna(subset=['TotalCharges'], inplace=True)
    
    y = LabelEncoder().fit_transform(churn['Churn'])
    X = churn.drop('Churn', axis=1)
    
    return X, y

# --- Preprocessing Pipeline Definition ---
def define_preprocessor(X: pd.DataFrame):
    """
    Defines the preprocessing steps for numerical and categorical features.
    """
    # Identify categorical and numerical features
    categorical_features = X.select_dtypes(include=['object']).columns
    numerical_features = X.select_dtypes(include=np.number).columns

    # Create preprocessing pipelines for numerical and categorical features
    numerical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    # Create a preprocessor using ColumnTransformer
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_features),
            ('cat', categorical_transformer, categorical_features)
        ],
        remainder='passthrough'  # Keep other columns not specified
    )
    return preprocessor

# --- Model Training, Evaluation, and MLflow Logging Function ---
def train_evaluate_log_model(model_name: str, classifier, X_full: pd.DataFrame, y_full: np.ndarray, model_params: dict, preprocessor: ColumnTransformer, n_splits_cv_param: int):
    """
    Trains and evaluates a given model using cross-validation, logs metrics and the model to MLflow.
    """
    with mlflow.start_run(run_name=f"{model_name} Training"):
        print(f"\n--- Training {model_name} with Cross-Validation ---")
        mlflow.log_params(model_params)

        pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                   ('classifier', classifier(**model_params))])

        # --- Perform Cross-Validation using scikit-learn's cross_validate ---
        # This is more concise and less error-prone than a manual loop.
        skf = StratifiedKFold(n_splits=n_splits_cv_param, shuffle=True, random_state=RANDOM_SEED)
        scoring_metrics = {
            'accuracy': 'accuracy',
            'precision': 'precision_weighted',
            'recall': 'recall_weighted',
            'f1': 'f1_weighted',
            'roc_auc': 'roc_auc'
        }
        
        cv_results = cross_validate(pipeline, X_full, y_full, cv=skf, scoring=scoring_metrics, return_train_score=False)

        # --- Log Mean CV Metrics ---
        print(f"  {model_name} CV Results:")
        for metric_name in scoring_metrics.keys():
            # The key in cv_results is 'test_{metric_key_from_scoring}'
            scores = cv_results[f'test_{metric_name}']
            mean_score = np.mean(scores)
            std_score = np.std(scores)
            
            # Log to MLflow
            mlflow.log_metric(f"mean_cv_{metric_name}", mean_score)
            mlflow.log_metric(f"std_cv_{metric_name}", std_score)
            
            # Print to console
            print(f"    Mean Test {metric_name}: {mean_score:.4f} (Std: {std_score:.4f})")

        # The previous implementation logged individual fold metrics.
        # cross_validate does not expose fold-by-fold metrics easily within a single call.
        # Logging the mean and std is generally sufficient and cleaner.
        
        print(f"  Fitting final {model_name} pipeline on full training data for logging...")
        full_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                         ('classifier', classifier(**model_params))])
        full_pipeline.fit(X_full, y_full)

        # Infer signature for MLflow logging
        example_input = X_full.sample(1, random_state=RANDOM_SEED)
        signature = infer_signature(example_input, full_pipeline.predict(example_input))

        # Log the final trained pipeline
        mlflow.sklearn.log_model(
            sk_model=full_pipeline, 
            artifact_path="final_model_pipeline",
            signature=signature,
            input_example=example_input
        )
        print(f"  Final {model_name} pipeline logged to: {mlflow.active_run().info.artifact_uri}/final_model_pipeline")

# --- Main Execution Function ---
def main():
    """
    Orchestrates the entire ML pipeline: data loading, preprocessing,
    and training/logging multiple models.
    """
    print("Starting Telco Churn Classification with CV and Model Logging...")

    # Get the directory where the current script is located
    base_dir = os.path.dirname(os.path.abspath(__file__))

    # --- Set Tracking URI for Model Registry Support ---
    # The default file-based store ('./mlruns') does not support the Model Registry.
    # To use registry features (like mlflow.register_model), we need a database-backed store.
    # Here, we use a local SQLite database file. MLflow will create 'mlflow.db' in the script's directory.
    # Artifacts will still be stored in a local 'mlruns' directory by default.
    db_path = os.path.join(base_dir, "mlflow.db")
    mlflow.set_tracking_uri(f"sqlite:///{db_path}")
    # Set and confirm MLflow experiment
    mlflow.set_experiment(MLFLOW_EXPERIMENT_NAME)
    exp = mlflow.get_experiment_by_name(MLFLOW_EXPERIMENT_NAME)

    if exp is not None:
        print(f"Logging to experiment ID: {exp.experiment_id}")
    else:
        raise ValueError(f"Experiment '{MLFLOW_EXPERIMENT_NAME}' not found or failed to create.")

    # Build the full path to the CSV file
    data_path = os.path.join(base_dir, 'churn_data', 'WA_Fn-UseC_-Telco-Customer-Churn.csv')

    # Load data
    X_full, y_full = load_data(data_path)
    print(f"Data loaded. X_full shape: {X_full.shape}, y_full shape: {y_full.shape}")

    # Define preprocessor
    preprocessor = define_preprocessor(X_full)
    print("Preprocessor defined.")

    # Define models to compare
    models_to_compare = [
        {
            "name": "Logistic Regression",
            "classifier": LogisticRegression,
            "params": {"solver": "liblinear", "random_state": RANDOM_SEED}
        },
        {
            "name": "Random Forest Classifier",
            "classifier": RandomForestClassifier,
            "params": {"n_estimators": 100, "random_state": RANDOM_SEED}
        },
        {
            "name": "Gradient Boosting Classifier",
            "classifier": GradientBoostingClassifier,
            "params": {"n_estimators": 100, "random_state": RANDOM_SEED}
        }
    ]

    # Run the full pipeline for each model
    for model_info in models_to_compare:
        train_evaluate_log_model(
            model_info["name"],
            model_info["classifier"],
            X_full,
            y_full,
            model_info["params"],
            preprocessor,
            N_SPLITS_CV
        )

    print("\nAll Telco Churn models processed with CV and models logged to MLflow.")
    print(f"To view results, run 'mlflow ui' in your terminal and navigate to the '{MLFLOW_EXPERIMENT_NAME}' experiment.")

# --- Guard to run main() when script is executed directly ---
if __name__ == "__main__":
    main()